# 优化器类

我们通过张量封装了梯度计算，通过层封装了前向传播。

现在，我们将引入优化器类（Optimizer）来封装反向传播。

-----------------

至此，我们完成了对人工神经元网络模型三条数据链的封装：
* 张量：梯度计算
* 层：前向传播
* 损失函数：前向传播和梯度计算的连接点
* 优化器：反向传播

In [61]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [62]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### 基础层

为了配合优化器自动化反向传播的目的，我们需要在层中添加一个parameters()函数，来返回本层需要参与反向传播的张量（各类参数和预测值）。

In [63]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### 基础损失函数

In [64]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

基础优化器是一个抽象类，定义了一个反向传播的接口。

优化器需要两个参数：
* parameters：各层参与反向传播的张量集合，这些张量都需要根据梯度进行更新
* lr：学习率

In [65]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def step(self):
        pass

## 数据

### 特征、标签

In [66]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

在线性层，权重和偏差需要参与反向传播，需要根据梯度进行更新。

In [67]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### 损失函数（平均平方差）

In [68]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

优化器（随机梯度下降）继承了基础优化器的接口，根据梯度随机下降的规则更新parameters中的所有张量。

In [69]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## 设置

### 学习率

In [70]:
LEARNING_RATE = 0.00001

## 验证

### 推理

In [71]:
layer = Linear(feature.size(), 1)
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: [43.05]


### 评估

In [72]:
loss = MSELoss()
error = loss(prediction, label)
print(f'error: {error}')

error: 14871.802500000002


## 训练

### 梯度计算

In [73]:
error.backward()

### 反向传播

完成梯度计算以后，我们可以方便地调用优化器的step()函数，完成一次迭代的反向传播。

在创建优化器的时候，我们需要提供每层所有需要更新的张量的集合。

In [74]:
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)
optimizer.step()
print(layer)

weight: [[0.5685359 0.641462 ]]
bias: [0.002439]


### 重新评估

In [75]:
prediction = layer(feature)
error = loss(prediction, label)
print(f'error: {error}')

error: 12503.020514375934
